In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
from matplotlib import pyplot as plt
import seaborn as sns

# Suppress FutureWarnings from pandas .interpolate()
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [ ]:
version = '250519'

In [ ]:
def expand_group(group):
    icu_stay_id = group['icu_stay_id'].iloc[0]
    min_offset = group['offset'].min()
    max_offset = group['offset'].max()
    full_range = pd.DataFrame({
        'icu_stay_id': icu_stay_id,
        'offset': np.arange(min_offset, max_offset + 1)
    })
    return full_range.merge(group, on=['icu_stay_id', 'offset'], how='left')

df = pd.read_csv(f'../data/blood_pressure_{version}.csv')[['icu_stay_id', 'offset', 'invasive_mbp']]

df = df.groupby('icu_stay_id', group_keys=False).apply(expand_group)

df = df.sort_values(by=['icu_stay_id', 'offset']).reset_index(drop=True)

len(df), len(df['icu_stay_id'].unique())

In [ ]:
df.head()

In [ ]:
def is_continuous(offset_series):
    return (offset_series.sort_values().diff().dropna() == 1).all()

# Check continuity for each icu_stay_id
continuity_check = df.groupby('icu_stay_id')['offset'].apply(is_continuous)

# Show ICU stay IDs where offset is not continuous
non_continuous_ids = continuity_check[~continuity_check].index.tolist()

len(non_continuous_ids)

In [ ]:
def lowpass(x, samplerate, fp, fs, gpass, gstop):
    fn = samplerate / 2   # Nyquist frequency
    wp = fp / fn          # Normalize passband edge frequency
    ws = fs / fn          # Normalize stopband edge frequency
    N, Wn = signal.buttord(wp, ws, gpass, gstop)  # Calculate Butterworth filter order and normalized cutoff
    b, a = signal.butter(N, Wn, "low")            # Compute numerator and denominator of the filter transfer function
    y = signal.filtfilt(b, a, x)                  # Apply zero-phase filtering
    return y

In [ ]:
list_id = df['icu_stay_id'].unique()

In [ ]:
pid = list_id[0]
plt.plot(df[df['icu_stay_id']==pid]['offset'], df[df['icu_stay_id']==pid]['invasive_mbp'])

In [ ]:
samplerate = 1/60
fpass = 0.15 * samplerate  # Passband edge frequency [Hz]
fstop = 0.40 * samplerate  # Stopband edge frequency [Hz]
gpass = 3   # Maximum loss in the passband [dB] - parameter for attenuation curve between fpass and fstop
gstop = 40  # Minimum attenuation in the stopband [dB] - parameter for attenuation curve between fpass and fstop

mbp_lpf = []
for icu_stay_id in list_id:
    # print(icu_stay_id)
    df_tmp = df[df['icu_stay_id'] == icu_stay_id]
    df_tmp = df_tmp.interpolate(limit_direction='both', inplace=False)
    tmp = lowpass(df_tmp['invasive_mbp'].values, samplerate, fpass, fstop, gpass, gstop)
    mbp_lpf = np.concatenate([mbp_lpf, tmp])


In [ ]:
df['invasive_mbp_lpf'] = mbp_lpf

In [ ]:
for i in range(10):
    patient_id = list_id[i+30]
    
    # Filter the DataFrame for the current patient
    patient_data = df[df['icu_stay_id'] == patient_id]
    
    # Plot invasive_sbp_lpf over offset
    plt.figure(figsize=(10, 6))
    plt.plot(patient_data['offset'], patient_data['invasive_mbp'], label=f'Patient {patient_id}')
    plt.plot(patient_data['offset'], patient_data['invasive_mbp_lpf'], label=f'Patient {patient_id}')
    plt.xlabel('Offset')
    plt.ylabel('Invasive MBP LPF')
    plt.title(f'Invasive MBP LPF over Offset for Patient {patient_id}')
    plt.legend()
    plt.show()

In [ ]:

df[['invasive_mbp', 'invasive_mbp_lpf']].isnull().sum()

In [ ]:
df.to_csv(f'../data/blood_pressure_filtered_{version}.csv', header=True, index=False)

In [ ]:
patient_id = list_id[38]

patient_data = df[df['icu_stay_id'] == patient_id].copy()

# Prepare data in long format for seaborn
plot_df = patient_data[['offset', 'invasive_mbp', 'invasive_mbp_lpf']].melt(
    id_vars='offset',
    var_name='Signal Type',
    value_name='MAP Value'
)

# Rename signal types for better readability
plot_df['Signal Type'] = plot_df['Signal Type'].map({
    'invasive_mbp': 'Raw MAP',
    'invasive_mbp_lpf': 'Filtered MAP'
})

palette = {'Raw MAP': '#97BBF5CC', 'Filtered MAP': '#4269D0'}

fig, ax = plt.subplots(figsize=(12, 8))
sns.lineplot(
    data=plot_df,
    x='offset',
    y='MAP Value',
    hue='Signal Type',
    palette=palette,
    linewidth=2.5,     # 👈 Increased line width
    ax=ax
)

#ax.set_ylim(0, 120)
ax.set_xticks([-30, 0, 30, 60, 90, 120])
ax.set_title('Comparison of Raw and Low-pass Filtered MAP', fontsize=24)
ax.set_xlabel('Time Offset (minutes)', fontsize=20)
ax.set_ylabel('Mean Arterial Pressure (mmHg)', fontsize=20)
ax.tick_params(axis='both', which='major', labelsize=18)
ax.legend(title='Signal Type', fontsize=16, title_fontsize=18)

plt.tight_layout()

fig.savefig(f'../output/map_filtered_vs_raw_patient_{version}.png', dpi=300)

plt.show()